# <center>Image Colorization Using CNN</center>


## Importing libraries


In [1]:
import os
import cv2
import keras
import numpy
import skimage
import matplotlib
import tensorflow

## Getting image from dataset and converting into array


In [2]:
X = []
for image_name in os.listdir('Dataset\Training_image/'):
    X.append(tensorflow.keras.utils.img_to_array(
        tensorflow.keras.utils.load_img('Dataset\Training_image/'+image_name, target_size=(256, 256))))
X = numpy.array(X, dtype=float)

## Setting Training and Testing data


In [ ]:
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain
Xtest = X[split:]
Xtest = 1.0/255*Xtest

## Defining CNN Model


In [ ]:
model = keras.models.Sequential()

### Input layer


In [ ]:
model.add(keras.layers.Conv2D(64, (3, 3), input_shape=(
    256, 256, 1), activation='relu', padding='same'))

### Hidden Layers


In [ ]:
model.add(keras.layers.Conv2D(
    64, (3, 3), activation='relu', padding='same', strides=2))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(
    128, (3, 3), activation='relu', padding='same', strides=2))

model.add(keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(
    256, (3, 3), activation='relu', padding='same', strides=2))

model.add(keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.UpSampling2D((2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.UpSampling2D((2, 2)))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(2, (3, 3), activation='tanh', padding='same'))

model.add(keras.layers.UpSampling2D((2, 2)))

### Compile model


model.compile(optimizer='rmsprop', loss='mse', metrics = ['accuracy'])

## Image transformation


In [ ]:
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True)

## Generate training data


In [ ]:
batch_size = 10


def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = skimage.color.rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        Y_batch = lab_batch[:, :, :, 1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

## Train model

In [ ]:
tensorboard = tensorflow.keras.callbacks.TensorBoard(log_dir="Training_Information/Logs")
trainedmodel = model.fit(image_a_b_gen(batch_size), callbacks=[tensorboard],epochs=500, steps_per_epoch=30)

## Show model accuracy

In [ ]:
matplotlib.pyplot.plot(trainedmodel.history['accuracy'])
matplotlib.pyplot.title('model accuracy')
matplotlib.pyplot.ylabel('accuracy')
matplotlib.pyplot.xlabel('epoch')
matplotlib.pyplot.legend(['train', 'test'], loc='upper left')
matplotlib.pyplot.show()